In [18]:
import matplotlib.pyplot as plt
import sklearn.ensemble as sk
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
df = pd.read_csv('train_file.csv')

In [20]:
df.head()

,ID,UsageClass,CheckoutType,CheckoutYear,CheckoutMonth,Checkouts,Title,Creator,Subjects,Publisher,PublicationYear,MaterialType
0,1,Physical,Horizon,2005,4,1,Tidal wave,NaN,"Tsunamis, Tsunamis Juvenile literature",NaN,NaN,BOOK
1,2,Physical,Horizon,2005,4,1,London holiday / Richard Peck.,"Peck, Richard, 1934-",NaN,"Viking,",1998.,BOOK
2,3,Physical,Horizon,2005,4,3,Cinco de Mayo : celebrating Hispanic pride / C...,"Gnojewski, Carol",Cinco de Mayo Mexican holiday History Juvenile...,"Enslow Publishers,",c2002.,BOOK
3,4,Physical,Horizon,2005,4,1,Annapolis,NaN,"War stories, Historical fiction, Domestic fict...",NaN,NaN,BOOK
4,5,Physical,Horizon,2005,4,1,As a man thinketh,NaN,Thought and thinking,NaN,NaN,BOOK


In [21]:
df = df.fillna('')

In [22]:
mapper = DataFrameMapper([
     ('Title', TfidfVectorizer()),
     ('Checkouts', None),
 ])

In [23]:
features = mapper.fit_transform(df)
categories = df['MaterialType']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
categories=le.fit_transform(categories)

MemoryError: 

In [13]:
x, x_test, y, y_test = train_test_split(features,categories,test_size=0.2,train_size=0.8, random_state = 0)

In [14]:
clf = sk.RandomForestClassifier(random_state=0)
clf.fit(x, y)

/home/khandalaryan/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [15]:
predicted = clf.predict(x_test)
print(classification_reportcation_report(y_test, predicted))

              precision    recall  f1-score   support

        BOOK       0.80      0.87      0.84      4376
          CR       0.50      0.10      0.17        20
       MIXED       1.00      0.84      0.91        76
       MUSIC       0.71      0.12      0.21        41
   SOUNDCASS       0.17      0.04      0.06       194
   SOUNDDISC       0.43      0.54      0.48       791
   VIDEOCASS       0.26      0.11      0.16       530
   VIDEODISC       0.47      0.34      0.40       303

   micro avg       0.71      0.71      0.71      6331
   macro avg       0.54      0.37      0.40      6331
weighted avg       0.68      0.71      0.68      6331



In [17]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test,predicted)
acc=(mat[0][0]+mat[1][1])/(mat[0][0]+mat[1][1]+mat[0][1]+mat[1][0])
print(acc)

0.9960917144346013


In [135]:
df.drop(['ID','CheckoutType','CheckoutYear','CheckoutMonth','Creator','Subjects','Publisher','PublicationYear','CheckoutType'],axis=1,inplace=True)

In [136]:
x = df['Title']

In [137]:
import string
import nltk
from nltk.corpus import stopwords
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Return the cleaned text as a list of words
    '''
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [138]:
from sklearn.feature_extraction.text import TfidfVectorizer
bow_transformer = TfidfVectorizer(analyzer=text_process).fit(x)
X = bow_transformer.transform(x)

In [139]:
y = df['MaterialType']

In [140]:
X

<31653x39279 sparse matrix of type '<class 'numpy.float64'>'
	with 165155 stored elements in Compressed Sparse Row format>

In [141]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(y)

In [142]:
y

array([0, 0, 0, ..., 7, 0, 2])

In [143]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [144]:
def fitpred(nb):
    nb.fit(X_train, y_train)
    preds = nb.predict(X_test)
    mat = confusion_matrix(y_test, preds)
    print(mat)
    acc=(mat[0][0]+mat[1][1])/(mat[0][0]+mat[1][1]+mat[0][1]+mat[1][0])
    print(acc)
    print('\n')
    print(classification_report(y_test, preds))

In [145]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
nb = MultinomialNB()
fitpred(nb)
nb2 = BernoulliNB()
fitpred(nb2)
clf = svm.SVC(gamma='scale')
fitpred(clf)
clf2 = xgb.XGBClassifier()
fitpred(clf2)
clf3 = DecisionTreeClassifier()
fitpred(clf3)

[[6576    0    0    0    0    3    0    0]
 [  25    0    0    0    0    0    0    0]
 [ 104    0    0    0    0    2    0    0]
 [  42    0    0    0    0   11    0    0]
 [ 323    0    0    0    0    2    0    0]
 [ 889    0    0    0    0  299    0    0]
 [ 800    0    0    0    0    3    0    0]
 [ 375    0    0    0    0    1    0   41]]
0.9962126950462051


              precision    recall  f1-score   support

           0       0.72      1.00      0.84      6579
           1       0.00      0.00      0.00        25
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        53
           4       0.00      0.00      0.00       325
           5       0.93      0.25      0.40      1188
           6       0.00      0.00      0.00       803
           7       1.00      0.10      0.18       417

   micro avg       0.73      0.73      0.73      9496
   macro avg       0.33      0.17      0.18      9496
weighted avg       0.66      0.73     

/home/khandalaryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/khandalaryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[6579    0    0    0    0    0    0    0]
 [  25    0    0    0    0    0    0    0]
 [ 106    0    0    0    0    0    0    0]
 [  53    0    0    0    0    0    0    0]
 [ 325    0    0    0    0    0    0    0]
 [1151    0    0    0    0   37    0    0]
 [ 803    0    0    0    0    0    0    0]
 [ 417    0    0    0    0    0    0    0]]
0.9962144155057541


              precision    recall  f1-score   support

           0       0.70      1.00      0.82      6579
           1       0.00      0.00      0.00        25
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        53
           4       0.00      0.00      0.00       325
           5       1.00      0.03      0.06      1188
           6       0.00      0.00      0.00       803
           7       0.00      0.00      0.00       417

   micro avg       0.70      0.70      0.70      9496
   macro avg       0.21      0.13      0.11      9496
weighted avg       0.61      0.70     

In [146]:
nbf = xgb.XGBClassifier()
nbf.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [147]:
test = pd.read_csv('test_file.csv')

In [148]:
X_t = test['Title']
X_t = bow_transformer.transform(X_t)

In [149]:
preds = nbf.predict(X_t)

In [150]:
preds = le.inverse_transform(preds)

In [151]:
preds

array(['BOOK', 'BOOK', 'SOUNDDISC', ..., 'BOOK', 'BOOK', 'SOUNDDISC'],
      dtype=object)

In [152]:
id_ = test['ID']
label = preds
data = { 'ID': id_, 'MaterialType': label}
submission = pd.DataFrame(data)
submission.head(25)

,ID,MaterialType
0,31654,BOOK
1,31655,BOOK
2,31656,SOUNDDISC
3,31657,BOOK
4,31658,BOOK
5,31659,BOOK
6,31660,BOOK
7,31661,BOOK
8,31662,BOOK
9,31663,BOOK


In [153]:
submission.to_csv('xgb.csv',index=False)